# Connexion au serveur MLFlow - Test et Configuration Locale

Ce notebook permet de **tester la connexion à un serveur MLFlow déployé localement via Docker sur un NAS/Serveur personnel**. Il est conçu pour vérifier que toutes les configurations sont correctes et que nous pouvons commencer à suivre nos expériences de machine learning
.
## Prérequis

Avant d'exécuter ce notebook, assurez-vous d'avoir :
- Un serveur MLFlow déployé localement via Docker sur votre NAS. Il sera important d'installer dans le conteneur la librairie boto3 pour que MLFlow puisse échanger avec un serveur de type AWS pour le stockage des artefacts.
- Un serveur MinIO (compatible S3) déployé localement via Docker pour le stockage des artefacts et qui remplacera les Amazon Web Services (AWS).
- Les informations de connexion nécessaires (URI de tracking, identifiants MinIO).
- Les ports 5000 (MLFlow) et 9000 (MinIO API) correctement exposés et/ou gérés par un reverse proxy (SWAG).

## Configuration des variables d'environnement dans Jupyter Notebook / Google Collab

Pour une connexion réussie, vous devez définir les variables d'environnement dans un fichier `.env` qui sera chargé via la librairie dotenv dans le répertoire courant. Ces variables seront utilisées par la bibliothèque MLflow pour communiquer avec vos services.

Voici les variables essentielles à configurer (urls locaux en guise d'exemple) :

```
MLFLOW_TRACKING_URI=http://mlflow:5000
MLFLOW_S3_ENDPOINT_URL=http://minio:9000
AWS_ACCESS_KEY_ID=minioadmin
AWS_SECRET_ACCESS_KEY=minioadminpassword
```

Remplacez les valeurs par vos propres informations de connexion.

Note : minio possède deux ports :
- 9001 pour accéder à l'interface web et gérer le stockage / nettoyage d'artefacts
- 9000 qui est l'api qui sera requêté pour l'envoi de buckets S3 par la librairie mlflow

## Étapes du test

Ce notebook effectue les opérations suivantes :
1. Chargement des variables d'environnement depuis le fichier `.env`
2. Configuration de MLFlow avec ces variables
3. Tentative de connexion au serveur MLFlow
4. Récupération de la liste des expériences existantes
5. Création d'une expérience si elle n'existe pas déjà
6. Enregistrement de métriques test pour vérifier le bon fonctionnement

## Résolution des problèmes courants

Notez que si vous utilisez des notebooks Google Collab, il est **nécessaire** que votre serveur MLFlow ainsi que l'API de MinIO soient accessibles via un lien public. Vous pouvez simplement ouvrir les ports de votre modem ou, pour plus de sécurité, configurer un reverse proxy (type SWAG pour la gestion et délivrance automatique de certificats SSL). Cette étape ne sera pas développée ici.

Notez également que si vous désactivez votre docker MLFlow et le réactivez (par exemple suite à un redémarrage), les expériences seront par défaut perdues. Pour qu'elles soient persistantes, il faudra rajouter dans le yaml du docker l'argument de commande ainsi que le volume suivants :
- ` - --backend-store-uri  ` \
  `    - sqlite:////mlflow/mlflow.db  # <-- pointe vers le volume monté `
- `      - ./mlflow-data:/mlflow  # Monte un volume local pour préserver mlflow.db`

Si vous rencontrez des erreurs lors de la connexion, vérifiez :
- Que l'URI de tracking est correct et accessible
- Que vos identifiants AWS sont valides (si applicable)
- Que le serveur MLFlow est bien en cours d'exécution
- Vos paramètres réseau (pare-feu, VPN, etc.)

In [ ]:
!pip install mlflow boto3 dotenv --quiet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
drive_folder=  "/content/drive/MyDrive/Colab_Notebooks/Project_7/notebooks"
os.chdir(drive_folder)
# os.listdir()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import mlflow
from dotenv import load_dotenv

# Charger les variables d'environnement depuis le fichier .env
load_dotenv(dotenv_path='../.env')

# Configuration de MLflow avec les variables d'environnement
mlflow_tracking_uri = os.getenv("MLFLOW_TRACKING_URI")
mlflow_s3_endpoint_url = os.getenv("MLFLOW_S3_ENDPOINT_URL")
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")

print(f"MLflow Tracking URI: {mlflow_tracking_uri}")

# Configuration explicite des identifiants AWS pour MLflow
os.environ["MLFLOW_S3_ENDPOINT_URL"] = mlflow_s3_endpoint_url
os.environ["AWS_ACCESS_KEY_ID"] = aws_access_key_id
os.environ["AWS_SECRET_ACCESS_KEY"] = aws_secret_access_key
print("Identifiants AWS configurés")

# Configuration de MLflow
try:
    print("Tentative de connexion à MLflow...")
    mlflow.set_tracking_uri(mlflow_tracking_uri)

    # Liste des expériences pour vérifier la connexion
    exps = mlflow.search_experiments()
    print(f"Connexion à MLflow réussie! Nombre d'expériences existantes: {len(exps)}")
    for exp in exps:
        print(f"  - {exp.name} (ID: {exp.experiment_id})")

    # Définir le nom de l'expérience
    experiment_name = "Demo PC"

    # Obtenir ou créer l'expérience
    experiment = mlflow.get_experiment_by_name(experiment_name)
    if experiment is None:
        experiment_id = mlflow.create_experiment(experiment_name)
        print(f"Nouvelle expérience créée: '{experiment_name}' (ID: {experiment_id})")
    else:
        experiment_id = experiment.experiment_id
        print(f"Expérience existante trouvée: '{experiment_name}' (ID: {experiment_id})")

    # Démarrer un simple run et enregistrer une métrique
    with mlflow.start_run(experiment_id=experiment_id, run_name="test_connexion_simple"):
        print("Démarrage d'un run MLflow de test...")
        mlflow.log_param("test_param", "valeur_test")
        mlflow.log_metric("test_metric", 1.0)
        print("Métrique de test enregistrée avec succès!")

    print(f"Test terminé avec succès. Vous pouvez vérifier les résultats sur: {mlflow_tracking_uri}")

except Exception as e:
    print(f"Erreur lors de la connexion à MLflow: {e}")
    print("\nConseils de débogage:")
    print("1. Vérifiez que l'URI de tracking est correct et accessible")
    print("2. Vérifiez que vos identifiants AWS sont valides si vous utilisez S3")
    print("3. Vérifiez que le serveur MLflow est en cours d'exécution")
    print("4. Vérifiez les paramètres réseau (pare-feu, VPN, etc.)")

MLflow Tracking URI: https://mlflow.greg-madman-nas.duckdns.org/
Identifiants AWS configurés
Tentative de connexion à MLflow...
Connexion à MLflow réussie! Nombre d'expériences existantes: 3
  - Demo PC (ID: 2)
  - OC Projet 7 (ID: 1)
  - Default (ID: 0)
Expérience existante trouvée: 'Demo PC' (ID: 2)
Démarrage d'un run MLflow de test...
Métrique de test enregistrée avec succès!
🏃 View run test_connexion_simple at: https://mlflow.greg-madman-nas.duckdns.org/#/experiments/2/runs/3127176c596441c48e34590d27494b56
🧪 View experiment at: https://mlflow.greg-madman-nas.duckdns.org/#/experiments/2
Test terminé avec succès. Vous pouvez vérifier les résultats sur: https://mlflow.greg-madman-nas.duckdns.org/


In [ ]:
import os
import mlflow
from mlflow.models import infer_signature
from mlflow import MlflowClient
from dotenv import load_dotenv

import pandas as pd
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Charger les variables d'environnement depuis le fichier .env
load_dotenv()

# Configuration de MLflow avec les variables d'environnement
mlflow_tracking_uri = os.getenv("MLFLOW_TRACKING_URI")
mlflow_s3_endpoint_url = os.getenv("MLFLOW_S3_ENDPOINT_URL")
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")

# Configuration explicite de MLflow
mlflow.set_tracking_uri(mlflow_tracking_uri)
print(f"MLflow Tracking URI: {mlflow_tracking_uri}")

# Configuration explicite des identifiants AWS
os.environ["MLFLOW_S3_ENDPOINT_URL"] = mlflow_s3_endpoint_url
os.environ["AWS_ACCESS_KEY_ID"] = aws_access_key_id
os.environ["AWS_SECRET_ACCESS_KEY"] = aws_secret_access_key
print("Identifiants AWS configurés")

# Create a new MLflow Experiment
mlflow.set_experiment("Demo PC")

# Define the output directory
output_dir = "./content/mlflow_test/"

# Create the output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Output directory created: {output_dir}")

MLflow Tracking URI: https://mlflow.greg-madman-nas.duckdns.org/
Identifiants AWS configurés


In [ ]:
# Fonction pour afficher les informations du run MLflow
def print_logged_info(r):
    print("")
    print("🏃 MLflow Run Infos :")
    tags = {k: v for k, v in r.data.tags.items() if not k.startswith("mlflow.")}
    artifacts = [f.path for f in MlflowClient().list_artifacts(r.info.run_id, "model")]
    print(f"⚗️ run_id: {r.info.run_id}")
    print(f"⚡ artifacts: {artifacts}")
    print(f"⚙️ params: {r.data.params}")
    print(f"📝 metrics: {r.data.metrics}")
    print(f"🏷️ tags: {tags}")
    print()

# Chargement du dataset Iris
print("Chargement du dataset Iris...")
iris = datasets.load_iris()
X, y = iris.data, iris.target

# Conversion en DataFrame pour une meilleure visualisation
iris_df = pd.DataFrame(data=np.c_[iris['data'], iris['target']],
                      columns=iris['feature_names'] + ['target'])
print(f"Aperçu du dataset:\n{iris_df.head()}")
print(f"Dimensions du dataset: {iris_df.shape}")

# Créer une figure pour visualiser la distribution des classes
plt.figure(figsize=(8, 6))
sns.countplot(x='target', data=iris_df)
plt.title('Distribution des classes du dataset Iris')
# Save the figure in the output directory
plt.savefig(os.path.join(output_dir, 'iris_class_distribution.png'))
plt.close()

# Division en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Taille de l'ensemble d'entraînement: {X_train.shape}")
print(f"Taille de l'ensemble de test: {X_test.shape}")

# Définition des hyperparamètres du modèle
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "random_state": 8888,
}

# Démarrer un run MLflow
with mlflow.start_run(run_name="iris_classification") as run:
    print(f"Démarrage du run MLflow avec l'ID: {run.info.run_id}")

    # Enregistrer les hyperparamètres
    mlflow.log_params(params)

    # Entraîner le modèle
    print("Entraînement du modèle de régression logistique...")
    lr = LogisticRegression(**params)
    lr.fit(X_train, y_train)

    # Prédire sur l'ensemble de test
    y_pred = lr.predict(X_test)

    # Calculer les métriques
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    # Enregistrer les métriques
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)

    print(f"Métriques: Accuracy={accuracy:.4f}, Precision={precision:.4f}, Recall={recall:.4f}, F1={f1:.4f}")

    # Générer une matrice de confusion
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=iris.target_names,
                yticklabels=iris.target_names)
    plt.title('Matrice de confusion')
    plt.xlabel('Prédiction')
    plt.ylabel('Réalité')
    plt.tight_layout()
    # Save the figure in the output directory
    plt.savefig(os.path.join(output_dir, 'confusion_matrix.png'))
    plt.close()

    # Enregistrer la matrice de confusion comme artifact
    mlflow.log_artifact(os.path.join(output_dir, "confusion_matrix.png"))
    mlflow.log_artifact(os.path.join(output_dir, "iris_class_distribution.png"))

    # Définir des tags pour ce run
    mlflow.set_tag("model_type", "LogisticRegression")
    mlflow.set_tag("dataset", "Iris")
    mlflow.set_tag("description", "Classification multiclasse des fleurs Iris")

    # Inférer la signature du modèle
    signature = infer_signature(X_train, lr.predict(X_train))

    # Enregistrer le modèle avec sa signature
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="model", # This is the path within the MLflow run artifact storage
        signature=signature,
        input_example=X_train[:5]
    )

    print(f"Modèle enregistré: {model_info.model_uri}")

# Récupérer et afficher les informations du run terminé
run_info = mlflow.get_run(run_id=run.info.run_id)
print_logged_info(run_info)

Chargement du dataset Iris...
Aperçu du dataset:
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target  
0     0.0  
1     0.0  
2     0.0  
3     0.0  
4     0.0  
Dimensions du dataset: (150, 5)
Taille de l'ensemble d'entraînement: (120, 4)
Taille de l'ensemble de test: (30, 4)
Démarrage du run MLflow avec l'ID: c9e768a9ca904c55a963f03568e43ecb
Entraînement du modèle de régression logistique...
Métriques: Accuracy=1.0000, Precision=1.0000, Recall=1.0000, F1=1.0000


2025/07/03 12:50:45 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Modèle enregistré: models:/m-abf8c8b98d6f4d60aa9898de7ee6a795
🏃 View run iris_classification at: https://mlflow.greg-madman-nas.duckdns.org/#/experiments/2/runs/c9e768a9ca904c55a963f03568e43ecb
🧪 View experiment at: https://mlflow.greg-madman-nas.duckdns.org/#/experiments/2

🏃 MLflow Run Infos :
⚗️ run_id: c9e768a9ca904c55a963f03568e43ecb
⚡ artifacts: ['model/MLmodel', 'model/conda.yaml', 'model/input_example.json', 'model/model.pkl', 'model/python_env.yaml', 'model/requirements.txt', 'model/serving_input_example.json']
⚙️ params: {'solver': 'lbfgs', 'max_iter': '1000', 'random_state': '8888'}
📝 metrics: {'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1_score': 1.0}
🏷️ tags: {'model_type': 'LogisticRegression', 'dataset': 'Iris', 'description': 'Classification multiclasse des fleurs Iris'}



In [ ]:
# %%
# Prompts user with the Run ID from MLflow run
# You can find run ID in the Tracking UI
while True:
    run_id = input('Please enter your 🏃 RUN ID : ')
    if run_id.strip() != '':
        break

artifact_path = "model"

# Download artifact via the tracking server
mlflow_artifact_uri = f"runs:/{run_id}/{artifact_path}"

try:
    # Specify the local directory to download artifacts to
    local_path = mlflow.artifacts.download_artifacts(mlflow_artifact_uri, dst_path=output_dir)
    # Load the model
    model = mlflow.sklearn.load_model(local_path)

    # If the model prints, everything works!
    print(f"🚀 Model : {model}")
except:
  print("❌ Invalid Run ID.")

Please enter your 🏃 RUN ID : c9e768a9ca904c55a963f03568e43ecb


🚀 Model : LogisticRegression(max_iter=1000, random_state=8888)
